# Simultaneous equation solver

## Support functions

In [ ]:
# Reads the equation strings and converts them into lists of various values...
def getLists(equationList):
    var, coef, const, varCount, eqCount = {}, [], [], 0, 0
    for e in equationList:
        e, i, varsFound, sign = e + "\\", 0, [], 1
        # If coefficient of a variable is to the right of "=", we will take it to the LHS, reversing its sign.
        # If constant term is to the left of "=", we will take it to the RHS, reversing its sign.
        
        # Going through the equation...
        while e[i] != "\\":
            coefValue = ""
            while e[i].isspace(): i = i + 1 # To traverse possible spaces before '-'.
            if e[i] == "-": coefValue, i = coefValue + "-", i + 1 # Negative sign detection.
            while e[i].isspace(): i = i + 1 # To traverse possible spaces after '-'.
            
            # Number encountered...
            if e[i].isnumeric():
                coefValue, i = coefValue + e[i], i + 1
                while e[i].isnumeric() and e[i] != "\\":
                    coefValue, i = coefValue + e[i], i + 1
            
            # Alphabet encountered (potential variable)...
            if e[i].isalpha():
                varName, i = e[i], i + 1
                while e[i].isalnum() and e[i] != "\\":
                    varName, i = varName + e[i], i + 1
                # If variable already encountered in equation...
                if varName in varsFound:
                    coef[var[varName]][-1] = coef[var[varName]][-1] + float(coefValue) * sign
                # If variable is newly encountered in the equation...
                else:
                    varsFound.append(varName)
                    # If the variable is newly encountered in the system...
                    if(varName not in var):
                        var[varName], varCount = varCount, varCount + 1
                        coef.append([])
                    # If no numerical coefficient specified...
                    if coefValue == "" or coefValue == "-": coefValue = coefValue + "1"
                    # Making sure zero constants are put where required...
                    l = len(coef[var[varName]])
                    while l < eqCount:
                        coef[var[varName]].append(0)
                        l = l + 1
                    coef[var[varName]].append(float(coefValue) * sign)
            
            # If a constant is identified...
            elif coefValue != "":
                # If a constant already exists in the equation...
                if "c" in varsFound:
                    const[-1] = const[-1] + float(coefValue) * -sign
                # If a constant hasn't been encountered before...
                else:
                    varsFound.append("c")
                    const.append(float(coefValue) * -sign)
            
            # If equal-to sign encountered, invert the sign variable...
            else:
                if e[i] == "=": sign = -1
                i = i + 1
        eqCount = eqCount + 1
        
        # Making sure zero constant sums are put where required...
        if len(const) < eqCount: const.append(0)
    return (coef, const, var)

In [ ]:
# Uses the lists of values from "getLists" and creates the necessary matrices...
def getMatrices(equationList):
    (coef, const, var) = getLists(equationList)
    nVar, nEq = len(coef), len(const)
    A = zeros((nEq, nVar))
    B = zeros((nEq, 1))
    for i in range(0, nEq):
        for j in range(0, nVar):
            try: A[i][j] = coef[j][i]
            except: A[i][j] = 0
    for i in range(0, nEq):
        B[i][0] = const[i]
    return (A, B, var)

## Main function

In [ ]:
# Uses the matrices from "getMatrices" and finds the solutions if they exists...

def solveSystem(equationList):
    (A, B, var) = getMatrices(equationList)
    A = matrix(A)
    B = matrix(B)
    print("Coefficient matrix:")
    print(A)
    print("Constant sum matrix:")
    print(B)
    try: linalg.inv(A)
    except:
        print("Inverse of coefficient matrix does not exist.")
        print("No solutions.")
    else: print("Inverse of coefficient matrix exists.")
    X = linalg.solve(A, B)
    for i in var:
        print("{0} = {1}".format(i, X[var[i], 0]))

# Cramer's rule

Solving equation systems using Cramer's rule. Hence, consider the following in the context of simultaneous equations.

In [26]:
import numpy as np
import copy as cp
def cramersRule(A, B):
# A is the matrix of coefficients.
# B is the column matrix of constant sums.
    X = []
    solutions = [] # Intended list of solutions.
    nVars = len(A[0]) # Length of a row => Number of variables.
    for i in range(0, nVars + 1):
        X.append(cp.deepcopy(A))
        X[i][:, i] = B
        solutions.append(np.linalg.det(X[i])/np.linalg.det(A))
    return solutions

## Example

$4x+y=6200$

$3x+3y=5600$

In [30]:
A = np.matrix([[4, 1], [3, 3]]) # Matrix of coefficients.
B = np.matrix([[6200], [5600]]) # Matrix of constant sums.
cramersRule(A, B)

[1444.4444444444437, 422.22222222222246]